In [ ]:
import pandas as pd
import sqlite3

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Exception as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Exception as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Exception as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

## Normalize Data

In [11]:
def normalize_data(filename):
    patient_data = []
    diabetic_pred = []
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        patient_data.append(line.strip().split(',')[0:9])

    for line in lines:
        diabetic_pred.append(line.strip().split(',')[0:10:9])
    
    return (patient_data, diabetic_pred)

patient_data, diabetic_pred = normalize_data('diabetes.csv')

db_file = 'normalized.db'

### Patient Data

In [16]:
conn = create_connection(db_file)
cur = conn.cursor()
sql_create_pd = ''' CREATE TABLE IF NOT EXISTS PatientData (
                                PatientID INTEGER NOT NULL PRIMARY KEY,
                                Pregnancies INTEGER NOT NULL,
                                Glucose FLOAT NOT NULL,
                                BloodPressure INTEGER NOT NULL,
                                SkinThickness FLOAT NOT NULL,
                                Insulin FLOAT NOT NULL,
                                BMI FLOAT NOT NULL,
                                Pedigree FLOAT NOT NULL,
                                Age INTEGER NOT NULL);
                                '''

create_table(conn,sql_create_pd, True)

In [30]:
conn = create_connection(db_file)
cur = conn.cursor()
sql_insert_pd = ''' INSERT OR IGNORE INTO PatientData (
                          Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, Pedigree, Age) VALUES (?,?,?,?,?,?,?,?);
                        '''

patient_info = []

for patient in patient_data[1:]:
    preg = patient[0]
    gluc = patient[1]
    bp = patient[2]
    skinthic = patient[3]
    ins = patient[4]
    bmi = patient[5]
    pedi = patient[6]
    age = patient[7]
    patient_info.append((preg, gluc, bp, skinthic, ins, bmi, pedi, age))

cur.executemany(sql_insert_pd, patient_info)
conn.commit()
conn.close()

In [32]:
conn = create_connection(db_file)
sql = ''' SELECT * FROM PatientData '''
pd.read_sql_query(sql,conn)

,PatientID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Pedigree,Age
0,1,1,6.0,148,72.0,35.0,0.0,33.6,0.627
1,2,2,1.0,85,66.0,29.0,0.0,26.6,0.351
2,3,3,8.0,183,64.0,0.0,0.0,23.3,0.672
3,4,4,1.0,89,66.0,23.0,94.0,28.1,0.167
4,5,5,0.0,137,40.0,35.0,168.0,43.1,2.288
...,...,...,...,...,...,...,...,...,...
763,764,764,10.0,101,76.0,48.0,180.0,32.9,0.171
764,765,765,2.0,122,70.0,27.0,0.0,36.8,0.340
765,766,766,5.0,121,72.0,23.0,112.0,26.2,0.245
766,767,767,1.0,126,60.0,0.0,0.0,30.1,0.349


### Diabetes Prediction

In [42]:
diabetic_pred[0:5]

[['\ufeffPatientID', 'Outcome'],
 ['1', '1'],
 ['2', '0'],
 ['3', '1'],
 ['4', '0']]

In [40]:
conn = create_connection(db_file)
cur = conn.cursor()
sql_create_dp = ''' CREATE TABLE IF NOT EXISTS DiabetesPrediction (
                                PatientID INTEGER NOT NULL PRIMARY KEY,
                                DiabetesPred INTEGER NOT NULL);
                                '''

create_table(conn,sql_create_dp, True)

In [49]:
conn = create_connection(db_file)
cur = conn.cursor()
sql_insert_dp = ''' INSERT OR IGNORE INTO DiabetesPrediction (
                      DiabetesPred) VALUES (?);
                    '''

patient_info = []

for patient in diabetic_pred[1:]:
    outcome = patient[1]
    patient_info.append((outcome,))

cur.executemany(sql_insert_dp, patient_info)
conn.commit()
conn.close()

In [50]:
conn = create_connection(db_file)
sql = ''' SELECT * FROM DiabetesPrediction '''
pd.read_sql_query(sql,conn)

,PatientID,DiabetesPred
0,1,1
1,2,0
2,3,1
3,4,0
4,5,1
...,...,...
763,764,0
764,765,0
765,766,0
766,767,1


### Pull Data

In [54]:
conn = create_connection(db_file)
cur = conn.cursor()
sql = ''' SELECT * FROM PatientData
        INNER JOIN DiabetesPrediction AS dp
        ON PatientData.PatientID = dp.PatientID '''
df = pd.read_sql_query(sql,conn)
df

,PatientID,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Pedigree,Age,PatientID,DiabetesPred
0,1,1,6.0,148,72.0,35.0,0.0,33.6,0.627,1,1
1,2,2,1.0,85,66.0,29.0,0.0,26.6,0.351,2,0
2,3,3,8.0,183,64.0,0.0,0.0,23.3,0.672,3,1
3,4,4,1.0,89,66.0,23.0,94.0,28.1,0.167,4,0
4,5,5,0.0,137,40.0,35.0,168.0,43.1,2.288,5,1
...,...,...,...,...,...,...,...,...,...,...,...
763,764,764,10.0,101,76.0,48.0,180.0,32.9,0.171,764,0
764,765,765,2.0,122,70.0,27.0,0.0,36.8,0.340,765,0
765,766,766,5.0,121,72.0,23.0,112.0,26.2,0.245,766,0
766,767,767,1.0,126,60.0,0.0,0.0,30.1,0.349,767,1
